In [1]:
import pandas as pd #data manpulation
import numpy as np
import matplotlib.pyplot as plt #data visualization
import seaborn as sns
import scipy

# 1) Cleaning + EDA + Standarization + Encoding 
import chardet #to detetmine dataset encoding

from sklearn.preprocessing import StandardScaler # scaling data so mean = 0 and std =  1 by ((x- u) / std)
from scipy import stats # for Box-Cox Transformation stats.boxcox(original_data)[0]
from mlxtend.preprocessing import minmax_scaling # for min_max scaling dont use it
#or
from sklearn.preprocessing import MinMaxScaler,normalize  #Tune input range to be 0 -> 1 by MinMaxScaler, same as above

#preprocessing: replace numeric and categorical
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder # label (ordinal or nominal)
from sklearn.preprocessing import OrdinalEncoder # object (no order)
from sklearn.preprocessing import OneHotEncoder # categorical 

# 2) feature enginerring
from sklearn.feature_selection import mutual_info_regression #mutal importance of feature
# a) numeric feature enginering
from sklearn.cluster import KMeans # Kmeans model for cluster feature
from IPython.display import display
from sklearn.decomposition import PCA # for loadings and adding new feature
# b) categorical feature enginering
from category_encoders import MEstimateEncoder # have m estimator


# 3) Data Modeling = train test split then model then get error
from sklearn.model_selection import train_test_split #also has cross_val_score (cv)
from sklearn.model_selection import RepeatedKFold # k fold cross validation it specify cv in corss_val_score dw
from sklearn.model_selection import cross_val_score # cv to get mean and see how much this feature affect the MAE error
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBRegressor 
from xgboost import XGBClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score # accuracy of classification
#for accurace of regression use model.score or cross_val_score

#pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# 4) Model summary = features importance and effect + sharp values
import eli5
from eli5.sklearn import PermutationImportance # feature importance
# from pdpbox import pdp, get_dataset, info_plots #for pdp plots
import shap  # package used to calculate Shap values
import time

C:\Users\ahmed\.conda\envs\machine_learning\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


# 1) Starting With Data

In [2]:
X_train = pd.read_csv('X_Train_Encoded_FeatEng_data_PCA.csv')
X_train_feat = pd.read_csv('X_Train_Encoded_FeatEng_data.csv')

X_valid = pd.read_csv('X_Valid_Encoded_FeatEng_data_PCA.csv')
X_valid_feat = pd.read_csv('X_Valid_Encoded_FeatEng_data.csv')

y_train = pd.read_csv('y_Train_Encoded_FeatEng_data.csv')
y_valid = pd.read_csv('y_Valid_Encoded_FeatEng_data.csv')
X_train

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,track_name
0,-1.713346,-0.608122,-0.810726,-0.917053,-0.022587,0.150899,0.209470,-0.533675,-0.169275,0.463205,-0.019191,My Way (feat. Monty)
1,1.498150,-0.315267,-0.110841,-0.232546,-1.603988,1.597978,1.301859,0.620655,0.257682,0.316783,-0.021975,For You
2,-1.421661,1.147806,0.508055,0.095405,-1.034742,-0.280619,-0.531308,1.416121,0.072877,-0.000304,-0.026184,Chop Suey!
3,-1.853383,1.442150,0.229241,-0.946114,1.473203,0.217755,-0.168214,-0.207889,0.046511,-0.055534,-0.023122,Besitos
4,-0.659000,-0.065930,-1.232195,-0.724984,0.306472,0.105805,-0.369093,-0.701480,-0.044706,-0.154725,-0.021423,Gone Country
...,...,...,...,...,...,...,...,...,...,...,...,...
239663,-1.336925,0.127536,-0.329964,-2.038618,-0.191035,-0.168323,0.111730,-0.126489,0.061404,0.529105,-0.019012,Pickin' Wildflowers
239664,0.257889,0.845882,-0.562845,1.949707,-1.045491,-0.421412,0.261850,0.817449,0.423044,0.459624,-0.027599,Breathe Life
239665,-1.568997,0.365075,-0.125769,-0.573321,-0.370082,0.349591,-0.350267,-0.444168,0.083481,-0.263795,-0.025077,Hope We Meet Again (feat. Chris Brown)
239666,-0.995574,1.348195,-0.687323,0.792199,-2.095140,0.267313,0.701306,-0.089891,-0.378536,-0.055190,-0.036126,Get Down


In [3]:
X_train.describe()

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11
count,2.396680e+05,2.396680e+05,2.396680e+05,2.396680e+05,2.396680e+05,2.396680e+05,2.396680e+05,2.396680e+05,2.396680e+05,2.396680e+05,2.396680e+05
mean,1.073220e-17,9.404015e-17,3.442012e-17,2.454768e-17,-6.937388e-17,2.195357e-17,-2.192393e-17,2.116793e-17,6.635731e-17,6.885506e-18,4.876925e-17
std,1.596842e+00,1.173511e+00,1.041311e+00,1.012885e+00,9.854681e-01,9.750351e-01,8.932372e-01,7.880958e-01,6.569809e-01,4.362891e-01,1.449459e-01
min,-3.267977e+00,-4.074961e+00,-3.164771e+00,-3.508244e+00,-3.599695e+00,-4.033462e+00,-2.938182e+00,-2.718215e+00,-3.043856e+00,-2.798669e+00,-4.241859e-02
25%,-1.146616e+00,-7.874729e-01,-7.445999e-01,-7.262544e-01,-6.894992e-01,-6.489228e-01,-6.229094e-01,-5.498631e-01,-3.875409e-01,-2.581395e-01,-2.521076e-02
50%,-4.127422e-01,-3.129411e-02,-8.491678e-02,-6.481729e-02,-5.938054e-03,1.648586e-02,-4.796009e-02,-6.115010e-02,3.222563e-02,9.095076e-03,-2.091835e-02
75%,7.623312e-01,7.823398e-01,6.517929e-01,6.853560e-01,6.657318e-01,6.884659e-01,5.472936e-01,4.889466e-01,4.099611e-01,2.803594e-01,-1.692808e-02
max,5.662876e+00,4.746446e+00,4.930081e+00,4.230779e+00,4.092541e+00,4.276617e+00,3.530913e+00,4.018448e+00,3.479278e+00,2.856733e+00,9.978693e-01


In [4]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 239668 entries, 0 to 239667
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   PC1         239668 non-null  float64
 1   PC2         239668 non-null  float64
 2   PC3         239668 non-null  float64
 3   PC4         239668 non-null  float64
 4   PC5         239668 non-null  float64
 5   PC6         239668 non-null  float64
 6   PC7         239668 non-null  float64
 7   PC8         239668 non-null  float64
 8   PC9         239668 non-null  float64
 9   PC10        239668 non-null  float64
 10  PC11        239668 non-null  float64
 11  track_name  239668 non-null  object 
dtypes: float64(11), object(1)
memory usage: 21.9+ MB


In [5]:
print(X_train.isnull().sum().sum(), X_valid.isnull().sum().sum()) # should be 0
print(y_train.isnull().sum().sum(), y_valid.isnull().sum().sum()) # should be 0

0 0
0 0


In [6]:
y_train

,Cluster
0,1
1,5
2,7
3,7
4,1
...,...
239663,7
239664,5
239665,7
239666,5


In [7]:
y_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 239668 entries, 0 to 239667
Data columns (total 1 columns):
 #   Column   Non-Null Count   Dtype
---  ------   --------------   -----
 0   Cluster  239668 non-null  int64
dtypes: int64(1)
memory usage: 1.8 MB


In [8]:
def make_mi_scores(X, y, discrete_features):
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

In [9]:
mi_scores = make_mi_scores(X_train.iloc[:, :-1], y_train['Cluster'], discrete_features=False)
mi_scores

PC1     1.381799
PC11    1.340068
PC2     1.300711
PC3     1.243362
PC7     1.241223
PC4     1.235453
PC5     1.195158
PC6     1.183837
PC8     1.156816
PC9     1.132399
PC10    1.120494
Name: MI Scores, dtype: float64

### Select Features:

(Will Do this Step After Modeling)

* Select Features Have MI > 0.2 or lets try 0.1 First

**Hint:**

##### There are 4 Types of Recomendation Systems: 

    1) Popularity
    2) Content Based Filtering
    3) Collaborative Based Filtering - have 2 types (Memory-Based or Model-Based)
    4) Hybird (could be 2 & 3 or any advanced combination)

**unpersonalized** is popularity & Content

**personalized** is Hybird & Collaborative (for each person)

Our Model will build Collaborative based Filtering (Model-Based: Which make model out of rating) but Memory-Based is memorizing relation btn users and items which is not Scalable so we will use Matrix Factorization (MF) Technique called (Funk MF in surprize library)



**Rating is explicit (not in our data) or implicit (how many mins the use listen to track)**

# 2) Modeling

* 1) Data is already Scaled
* 2) ML Algorithm: 
    * A) Prepare Data
    * B) Model (Chose Appropriate Model) + Predictions
    * C) Evaluate Model **(According To Some Metric)** + Hyperparameters Tuning
    * D) Experiments

In [9]:
import math
import random
from nltk.corpus import stopwords # stop words for text analysis (content filetering)
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import TfidfVectorizer # plan Text -> Weighted vector (content filtering)
from sklearn.metrics.pairwise import cosine_similarity # to calc similar vectors (content filtering)
from scipy.sparse.linalg import svds # Matrix Factorization (collaborative filtering)

In [10]:
# More Imports
from ast import literal_eval # Content Filtering
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer # Content Filtering
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity # Content Filtering
from nltk.stem.snowball import SnowballStemmer # Content Filtering
from nltk.stem.wordnet import WordNetLemmatizer # Content Filtering
from nltk.corpus import wordnet # Content Filtering

from surprise import Reader, Dataset, SVD # Collaborative Filtering
from surprise.model_selection import cross_validate # Collaborative Filtering

# import warnings; warnings.simplefilter('ignore')

#### A) Prepare Data

In [11]:
train = X_train.copy()
train_feat = X_train_feat.copy()

train['cluster'] = y_train
train_feat['cluster'] = y_train

train

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,track_name,cluster
0,-1.713346,-0.608122,-0.810726,-0.917053,-0.022587,0.150899,0.209470,-0.533675,-0.169275,0.463205,-0.019191,My Way (feat. Monty),1
1,1.498150,-0.315267,-0.110841,-0.232546,-1.603988,1.597978,1.301859,0.620655,0.257682,0.316783,-0.021975,For You,5
2,-1.421661,1.147806,0.508055,0.095405,-1.034742,-0.280619,-0.531308,1.416121,0.072877,-0.000304,-0.026184,Chop Suey!,7
3,-1.853383,1.442150,0.229241,-0.946114,1.473203,0.217755,-0.168214,-0.207889,0.046511,-0.055534,-0.023122,Besitos,7
4,-0.659000,-0.065930,-1.232195,-0.724984,0.306472,0.105805,-0.369093,-0.701480,-0.044706,-0.154725,-0.021423,Gone Country,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
239663,-1.336925,0.127536,-0.329964,-2.038618,-0.191035,-0.168323,0.111730,-0.126489,0.061404,0.529105,-0.019012,Pickin' Wildflowers,7
239664,0.257889,0.845882,-0.562845,1.949707,-1.045491,-0.421412,0.261850,0.817449,0.423044,0.459624,-0.027599,Breathe Life,5
239665,-1.568997,0.365075,-0.125769,-0.573321,-0.370082,0.349591,-0.350267,-0.444168,0.083481,-0.263795,-0.025077,Hope We Meet Again (feat. Chris Brown),7
239666,-0.995574,1.348195,-0.687323,0.792199,-2.095140,0.267313,0.701306,-0.089891,-0.378536,-0.055190,-0.036126,Get Down,5


In [12]:
valid = X_valid.copy()
valid_feat = X_valid_feat.copy()

valid['cluster'] = y_valid
valid_feat['cluster'] = y_valid

valid

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,track_name,cluster
0,-1.649865,-0.220830,0.397776,-1.077607,0.609772,-2.549943,-0.385099,1.444860,-0.016298,0.114610,-0.016049,Something New,7
1,-0.547524,0.000378,-0.343981,-0.268869,0.145539,0.221780,-0.303617,-0.726333,0.185589,-0.436863,-0.023285,Lay It All on Me (feat. Ed Sheeran),1
2,0.713585,-0.046390,-2.195066,0.327535,0.282330,-0.435668,2.040728,-0.881134,0.397650,0.224568,-0.025403,Dreams - 2004 Remaster,5
3,0.040314,-0.277041,-0.960165,-1.097837,0.292432,0.196906,-0.221858,-0.272947,0.230100,-0.028153,-0.018156,Austin,1
4,1.435434,2.019428,-2.076450,0.863690,0.719980,-0.759401,0.102136,0.020102,0.725589,0.374487,-0.026716,Even When It Hurts (Praise Song),5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26625,2.824693,-1.455259,-0.826965,0.054395,1.055844,-0.900850,-0.347412,-0.897734,0.415097,-0.508863,-0.014308,Beauty,4
26626,0.183480,-1.051613,0.449503,-1.216589,0.860882,-0.276759,0.503136,-0.602520,-1.582018,-0.156661,-0.017339,Erase Your Social,1
26627,-1.181954,-1.243043,-0.393856,-0.228508,-0.413298,-0.941502,-0.078567,-0.141413,-0.097394,-0.025239,-0.019293,All My Chains,2
26628,0.084936,2.078412,-0.671137,0.208095,0.494825,-0.220394,-2.500711,-0.173989,0.109927,0.237801,-0.023829,When I Go Down,7


In [13]:
df = pd.concat([train, valid], axis=0)
df_feat = pd.concat([train_feat, valid_feat], axis=0)
# df.to_csv('train_valid_pca.csv', index=False)
# df_feat.to_csv('train_valid_feat.csv', index=False)
df

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,track_name,cluster
0,-1.713346,-0.608122,-0.810726,-0.917053,-0.022587,0.150899,0.209470,-0.533675,-0.169275,0.463205,-0.019191,My Way (feat. Monty),1
1,1.498150,-0.315267,-0.110841,-0.232546,-1.603988,1.597978,1.301859,0.620655,0.257682,0.316783,-0.021975,For You,5
2,-1.421661,1.147806,0.508055,0.095405,-1.034742,-0.280619,-0.531308,1.416121,0.072877,-0.000304,-0.026184,Chop Suey!,7
3,-1.853383,1.442150,0.229241,-0.946114,1.473203,0.217755,-0.168214,-0.207889,0.046511,-0.055534,-0.023122,Besitos,7
4,-0.659000,-0.065930,-1.232195,-0.724984,0.306472,0.105805,-0.369093,-0.701480,-0.044706,-0.154725,-0.021423,Gone Country,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26625,2.824693,-1.455259,-0.826965,0.054395,1.055844,-0.900850,-0.347412,-0.897734,0.415097,-0.508863,-0.014308,Beauty,4
26626,0.183480,-1.051613,0.449503,-1.216589,0.860882,-0.276759,0.503136,-0.602520,-1.582018,-0.156661,-0.017339,Erase Your Social,1
26627,-1.181954,-1.243043,-0.393856,-0.228508,-0.413298,-0.941502,-0.078567,-0.141413,-0.097394,-0.025239,-0.019293,All My Chains,2
26628,0.084936,2.078412,-0.671137,0.208095,0.494825,-0.220394,-2.500711,-0.173989,0.109927,0.237801,-0.023829,When I Go Down,7


In [14]:
columns = df.columns
cat_index = []
index = 0
for col in columns:
    if df[col].dtype == 'object':
        cat_index.append(index)
    index = index + 1
cat_index

[11]

In [15]:
df['track_name'].nunique()

76268

### Content Filtering

In [16]:
import tqdm as tqdm # colab
from tqdm import tqdm_notebook

class SpotifyRecommender():
    def __init__(self, rec_data):
        #our class should understand which data to work with
        self.rec_data_ = rec_data.copy()
    
    #if we need to change data
    def change_data(self, rec_data):
        self.rec_data_ = rec_data

    def _get_similar_items_to_user_profile(self, song_id, res_data, topn=1000):
        res_data = res_data.drop('track_name', axis=1).copy()
        #Computes the cosine similarity between the user profile and all item profiles
        series_1 = self.rec_data_.iloc[song_id].copy()
        series_1 = series_1.drop(labels = ['track_name'])
        cosine_similarities = cosine_similarity(np.array(series_1).reshape(1, -1), res_data)
        #Gets the top similar items
        similar_indices = cosine_similarities.argsort().flatten()[-topn:]
        #Sort the similar items by similarity
        similar_items = sorted([(i, self.rec_data_.iloc[i, cat_index[0]], cosine_similarities[0,i]) for i in similar_indices], key=lambda x: -x[2])
        similar_items_df = pd.DataFrame(similar_items, columns=['index', 'track_name', 'Similarity'])
        similar_items_df = similar_items_df.drop_duplicates(subset = ['track_name'])
        return similar_items_df
    
    #function which returns recommendations, we can also choose the amount of songs to be recommended
    def get_recommendations_knn(self, song_name, amount=1):
        #choosing the data for our song
        try:
            song = self.rec_data_[(self.rec_data_.track_name.str.lower() == song_name.lower())].head(1).values[0] # vector of first time seen song
        except:
            print("This Song Doesn't Exsist")
            return
        #dropping the data with our song
        res_data = self.rec_data_[self.rec_data_.track_name.str.lower() != song_name.lower()]
        distances = []
        for r_song in tqdm_notebook(res_data.values):
            dist = 0
            for col in np.arange(len(res_data.columns)):
                #indeces of non-numerical columns
                if not col in [cat_index]:
                    #calculating the manhettan distances for each numerical feature
                    dist = dist + np.absolute(float(song[col]) - float(r_song[col]))
            distances.append(dist)
        res_data['distance'] = distances
#         sorting our data to be ascending by 'distance' feature
        res_data = res_data.sort_values('distance')
        res_data = res_data.drop_duplicates(subset = ['track_name'])
        columns = ['track_name', 'distance'] # name -> uri
        # mapping
        return res_data[columns][:amount]

    def get_recommendations_cosine(self, song_name, amount=1):
        distances = set()
        #choosing the data for our song
        try:
            index = self.rec_data_[(self.rec_data_.track_name.str.lower() == song_name.lower())].head(1).index.tolist() # vector of first time seen song
            index = index[0]
        except:
            print("This Song Doesn't Exsist")
            return
        #dropping the data with our song
        res_data = self.rec_data_[self.rec_data_.track_name.str.lower() != song_name.lower()]
        return self._get_similar_items_to_user_profile(index, res_data, topn=amount)

In [18]:
recomender = SpotifyRecommender(df)

In [19]:
song_name = "It Wasn't Me"
song_name_index = np.where(df['track_name'] == song_name)[0][0]
# print(song_name_index) # 7788
result_knn_pca = recomender.get_recommendations_knn(song_name, 10)
result_knn_pca

This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/266212 [00:00<?, ?it/s]


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,track_name,distance
117094,Si No Le Contesto,2.010052
7453,2AM.,2.102285
1048,Slow Down,2.317497
169116,Vamos A Pasarla Bien,2.462788
182850,Step Yo Game Up,2.493003
215413,Get Away,2.558907
23409,That's All I Need,2.649503
29913,Three Plus Five,2.660622
29251,I Hate This Part,2.671089
4788,Bitch Better Have My Money,2.678596


In [20]:
indecies = result_knn_pca.index
indecies = indecies.insert(0, song_name_index)
df.iloc[indecies, :]

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,track_name,cluster
7788,-0.961747,-0.669614,0.645421,1.300223,-0.602274,1.176519,-0.332135,-1.375426,0.025771,0.640844,-0.022192,It Wasn't Me,0
117094,-1.159830,-0.452340,0.577508,1.030507,-0.816131,1.367529,-0.304843,-1.406027,-0.212377,0.088007,-0.025516,Si No Le Contesto,0
7453,2.015134,0.864143,-1.224981,0.092307,1.410066,0.245370,-0.930781,0.176471,1.760012,-0.555476,-0.018268,Fooling Yourself (The Angry Young Man),4
1048,-0.260709,-0.768853,0.723438,1.119050,-0.579561,0.675375,-0.389339,-1.155696,0.346286,0.505502,-0.020810,Slow Down,0
169116,-0.981303,-0.180814,0.552025,1.374323,-0.249544,1.068379,-0.543728,-1.483229,-0.297562,-0.038796,-0.025892,Vamos A Pasarla Bien,0
182850,-1.295035,-0.611060,0.184366,1.648353,-0.461692,0.571261,-0.662373,-1.343262,-0.032260,0.516112,-0.023162,Step Yo Game Up,0
215413,0.189078,-0.759359,0.707972,0.838693,-0.739231,1.169544,-0.365821,-1.258179,-0.019846,0.187292,-0.021970,Get Away,0
23409,-1.086428,0.497530,-0.586774,0.286393,-0.913567,-0.210866,1.013915,-0.251402,-0.187728,0.143413,0.970801,We Found Love,5
29913,-0.173083,-0.850777,0.445380,1.681640,-0.297055,0.959571,-0.431622,-1.274143,0.303496,0.533747,-0.020617,Three Plus Five,0
29251,-0.789933,-0.016043,0.542312,0.353114,-0.842665,0.879692,-0.529465,-1.399862,0.016143,0.615460,-0.023684,I Hate This Part,0


In [21]:
result_cosine_pca = recomender.get_recommendations_cosine("It Wasn't Me", 10) # Cosine PCA RESULT
result_cosine_pca

,index,track_name,Similarity
0,117059,Go Flex,0.959989
1,238367,Hotel,0.959989
2,30912,Twisted Transistor,0.959989
3,247044,The World (Is Going Up In Flames),0.946886
4,98466,I Do Too,0.946886
5,14238,Vibes,0.946886
6,209328,Dude (Looks Like A Lady),0.946886
7,177535,At The End Of The Day,0.946886
8,134926,Oracle,0.946886
9,198038,There Goes My Life,0.946886


In [22]:
# out = np.concatenate(indecies.values).ravel()
# list(out[0:3])

In [23]:
indecies = pd.DataFrame(result_cosine_pca['index'].values)
# print(indecies)
indecies.loc[len(indecies)] = song_name_index
# swap
trash = int(indecies.loc[0].values)
indecies.loc[0] =  int(indecies.loc[len(indecies) - 1].values)
indecies.loc[len(indecies) - 1] = trash
indices = list(np.concatenate(indecies.values).ravel())
df.iloc[indices, :]

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,track_name,cluster
7788,-0.961747,-0.669614,0.645421,1.300223,-0.602274,1.176519,-0.332135,-1.375426,0.025771,0.640844,-0.022192,It Wasn't Me,0
238367,-0.733653,-1.261975,0.844291,1.663898,-0.569539,-0.131308,-0.512175,-0.046672,0.679118,0.345899,-0.018440,Hotel,2
30912,-1.218626,1.354298,-0.357120,0.377213,-2.359593,0.331059,0.654336,0.952410,-0.391956,0.038158,-0.033913,Twisted Transistor,5
7376,0.689834,0.393752,-0.792393,0.968234,-2.118416,0.908384,0.669187,-0.100483,0.392206,0.227531,-0.029477,The World (Is Going Up In Flames),5
98466,1.523973,-0.065759,1.803666,-1.109235,-1.806017,0.360002,0.536913,-0.070088,-0.780260,-0.138505,-0.024656,I Do Too,4
14238,-0.452522,-0.495705,0.940153,1.068402,-0.452682,0.099452,-1.454549,2.065698,-0.393803,-0.536212,-0.018469,Vibes,2
209328,-1.078637,0.284451,-1.326780,-1.337778,0.432722,0.104787,-0.792036,0.524674,-1.550754,-0.644170,-0.022098,Dude (Looks Like A Lady),1
177535,1.957848,-1.012733,2.344640,1.347848,2.813180,-0.654711,0.135448,-0.001268,-0.585494,-0.590247,-0.013390,At The End Of The Day,4
134926,-0.646339,0.713287,-0.611996,-0.362502,-1.473650,-0.550546,1.683207,0.037059,-0.112661,0.167605,-0.030783,Oracle,5
198038,0.012835,0.597094,-0.336193,-0.155476,1.741136,0.020070,-0.207626,-1.209164,0.830660,0.464351,0.981549,There Goes My Life,1


PCA Data:

knn (same cluster as Test_data (song)) is better than cosine_similarites

---

---

In [24]:
recomender_feat = SpotifyRecommender(df_feat)

In [25]:
result_knn_feat = recomender_feat.get_recommendations_knn("It Wasn't Me", 10)
result_knn_feat

This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/266212 [00:00<?, ?it/s]


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,track_name,distance
117094,Si No Le Contesto,1.440958
14242,2AM.,1.978751
87629,NA B YA,2.230109
88771,Bitch Better Have My Money,2.315350
95625,Traicionera,2.330268
30718,I've Been Going Through Things,2.382313
182850,Step Yo Game Up,2.399891
68024,That's All I Need,2.432702
169116,Vamos A Pasarla Bien,2.458134
106598,Walls,2.461977


In [26]:
indecies = result_knn_feat.index
indecies = indecies.insert(0, song_name_index)
df_feat.iloc[indecies, :] # Recall@5

,collaborative,track_duration_ms,Popularity,Acousticness,Danceability,Energy,Instrumentalness,Liveness,Loudness,Speechiness,Tempo,track_name,cluster
7788,0,0.018799,-1.231488,-0.643839,1.556256,-0.137360,-0.618374,1.152887,0.821745,-0.116920,-0.931767,It Wasn't Me,0
117094,0,-0.102948,-1.231488,-0.459685,1.524744,0.524917,-0.618374,1.067837,0.829138,-0.455024,-0.921048,Si No Le Contesto,0
14242,0,0.213140,-1.231488,-0.660033,1.524744,-0.559030,-0.618374,0.778666,0.221876,-0.149364,-0.623267,2AM.,0
87629,0,0.246598,-1.231488,-0.664552,1.581465,0.009121,-0.607494,1.322988,-0.216347,-0.074230,-0.383623,NA B YA,0
88771,0,-0.148134,-1.231488,-0.663422,1.102487,0.423612,-0.615797,0.676606,0.692362,-0.274019,-0.583015,Bitch Better Have My Money,0
95625,0,0.035875,-1.231488,-0.320345,1.070976,0.157445,-0.618374,0.353414,0.708492,-0.244989,-1.100584,Traicionera,0
30718,0,0.926712,-1.231488,-0.664552,0.919720,0.088275,-0.618374,0.795676,0.634223,-0.152779,-0.942691,I've Been Going Through Things,0
182850,0,0.681548,-1.231488,-0.819708,1.751628,0.088275,-0.618374,0.770161,0.864089,0.248506,-0.581998,Step Yo Game Up,0
68024,0,-0.276196,-1.231488,-0.789204,1.001650,0.102160,-0.618374,1.425048,0.217508,0.081161,-1.055504,That's All I Need,0
169116,0,0.592482,-1.231488,-0.278919,1.373488,0.476634,-0.618374,1.271958,0.594232,-0.265481,-0.704142,Vamos A Pasarla Bien,0


In [27]:
result_cosine_feat = recomender_feat.get_recommendations_cosine("It Wasn't Me", 10) # Cosine Feature RESULT
result_cosine_feat

,index,track_name,Similarity
0,117059,Go Flex,0.959987
1,30912,Twisted Transistor,0.959987
2,238367,Hotel,0.959987
3,247044,The World (Is Going Up In Flames),0.946882
4,209328,Dude (Looks Like A Lady),0.946882
5,177535,At The End Of The Day,0.946882
6,40419,Rescue Me,0.946882
7,21730,What'd I Say,0.946882
8,198038,There Goes My Life,0.946882
9,14238,Vibes,0.946882


In [28]:
indecies = pd.DataFrame(result_cosine_feat['index'].values)
# print(indecies)
indecies.loc[len(indecies)] = song_name_index
# swap
trash = int(indecies.loc[0].values)
indecies.loc[0] =  int(indecies.loc[len(indecies) - 1].values)
indecies.loc[len(indecies) - 1] = trash
indices = list(np.concatenate(indecies.values).ravel())
df_feat.iloc[indices, :]

,collaborative,track_duration_ms,Popularity,Acousticness,Danceability,Energy,Instrumentalness,Liveness,Loudness,Speechiness,Tempo,track_name,cluster
7788,0,0.018799,-1.231488,-0.643839,1.556256,-0.137360,-0.618374,1.152887,0.821745,-0.116920,-0.931767,It Wasn't Me,0
30912,0,-0.818410,-1.196997,-0.853489,-0.164282,1.478211,1.812602,-0.827934,1.260304,-0.248405,-0.305093,Twisted Transistor,5
238367,0,-0.156856,-1.231488,-0.836392,1.096185,-0.257565,-0.618374,0.319394,0.015199,1.568477,-0.587898,Hotel,2
7376,0,-0.506777,-1.231488,-0.586220,0.686533,-0.166038,1.573489,-0.764147,-0.230798,-0.883630,-1.289667,The World (Is Going Up In Flames),5
209328,0,0.715972,1.182839,0.783076,-0.416375,1.474347,-0.573607,-1.120507,1.370868,-0.552357,0.272179,Dude (Looks Like A Lady),1
177535,0,0.732859,0.320579,2.263090,-0.321840,-1.222017,-0.618374,2.410781,-1.843890,2.065388,0.079781,At The End Of The Day,4
40419,0,2.385003,-0.438209,-0.852939,-0.378561,1.182813,1.812602,-0.522604,0.601625,-0.272311,0.058308,Rescue Me,5
21730,0,2.308534,-0.541680,2.116218,-0.397468,0.329931,0.482871,-0.690153,0.125091,-0.326954,-0.841619,What'd I Say,5
198038,1,1.300442,0.768954,-0.728196,-0.183189,-0.747839,-0.610111,1.042322,-0.369591,-0.781175,0.575851,There Goes My Life,1
14238,0,-0.002528,-1.231488,0.440375,-0.914260,0.875568,-0.618374,-0.582139,0.389235,2.065388,-1.031695,Vibes,2


Feature Data:

knn behaves better than Cosine Simalrity however:

**Note: Cosine Similarites in both PCA and Feature Data have Same Result !!!!!!!!!!!!!!**

Result of Expirements with Clusters:

So Feature_Data + knn method is the best determining method (as its more acurate than knn on PCA Components)

---

##### Same Features for same track_name

In [29]:
df_feat[df_feat['track_name'] == "It Wasn't Me"]

,collaborative,track_duration_ms,Popularity,Acousticness,Danceability,Energy,Instrumentalness,Liveness,Loudness,Speechiness,Tempo,track_name,cluster
7788,0,0.018799,-1.231488,-0.643839,1.556256,-0.13736,-0.618374,1.152887,0.821745,-0.11692,-0.931767,It Wasn't Me,0
11902,0,0.018799,-1.231488,-0.643839,1.556256,-0.13736,-0.618374,1.152887,0.821745,-0.11692,-0.931767,It Wasn't Me,0
13227,0,0.018799,-1.231488,-0.643839,1.556256,-0.13736,-0.618374,1.152887,0.821745,-0.11692,-0.931767,It Wasn't Me,0
14012,0,0.018799,-1.231488,-0.643839,1.556256,-0.13736,-0.618374,1.152887,0.821745,-0.11692,-0.931767,It Wasn't Me,0
14499,0,0.018799,-1.231488,-0.643839,1.556256,-0.13736,-0.618374,1.152887,0.821745,-0.11692,-0.931767,It Wasn't Me,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16215,0,0.018799,-1.231488,-0.643839,1.556256,-0.13736,-0.618374,1.152887,0.821745,-0.11692,-0.931767,It Wasn't Me,0
20439,0,0.018799,-1.231488,-0.643839,1.556256,-0.13736,-0.618374,1.152887,0.821745,-0.11692,-0.931767,It Wasn't Me,0
21637,0,0.018799,-1.231488,-0.643839,1.556256,-0.13736,-0.618374,1.152887,0.821745,-0.11692,-0.931767,It Wasn't Me,0
24369,0,0.018799,-1.231488,-0.643839,1.556256,-0.13736,-0.618374,1.152887,0.821745,-0.11692,-0.931767,It Wasn't Me,0


In [30]:
df[df['track_name'] == "It Wasn't Me"]

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,track_name,cluster
7788,-0.961747,-0.669614,0.645421,1.300223,-0.602274,1.176519,-0.332135,-1.375426,0.025771,0.640844,-0.022192,It Wasn't Me,0
11902,-0.961747,-0.669614,0.645421,1.300223,-0.602274,1.176519,-0.332135,-1.375426,0.025771,0.640844,-0.022192,It Wasn't Me,0
13227,-0.961747,-0.669614,0.645421,1.300223,-0.602274,1.176519,-0.332135,-1.375426,0.025771,0.640844,-0.022192,It Wasn't Me,0
14012,-0.961747,-0.669614,0.645421,1.300223,-0.602274,1.176519,-0.332135,-1.375426,0.025771,0.640844,-0.022192,It Wasn't Me,0
14499,-0.961747,-0.669614,0.645421,1.300223,-0.602274,1.176519,-0.332135,-1.375426,0.025771,0.640844,-0.022192,It Wasn't Me,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16215,-0.961747,-0.669614,0.645421,1.300223,-0.602274,1.176519,-0.332135,-1.375426,0.025771,0.640844,-0.022192,It Wasn't Me,0
20439,-0.961747,-0.669614,0.645421,1.300223,-0.602274,1.176519,-0.332135,-1.375426,0.025771,0.640844,-0.022192,It Wasn't Me,0
21637,-0.961747,-0.669614,0.645421,1.300223,-0.602274,1.176519,-0.332135,-1.375426,0.025771,0.640844,-0.022192,It Wasn't Me,0
24369,-0.961747,-0.669614,0.645421,1.300223,-0.602274,1.176519,-0.332135,-1.375426,0.025771,0.640844,-0.022192,It Wasn't Me,0


#### Evaluate model

In [ ]:
#Top-N accuracy metrics consts
EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS = 100

class ModelEvaluator:


    def get_not_interacted_items_sample(self, person_id, sample_size, seed=42):
        interacted_items = get_items_interacted(person_id, interactions_full_indexed_df)
        all_items = set(articles_df['contentId'])
        non_interacted_items = all_items - interacted_items

        random.seed(seed)
        non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
        return set(non_interacted_items_sample)

    def _verify_hit_top_n(self, item_id, recommended_items, topn):        
            try:
                index = next(i for i, c in enumerate(recommended_items) if c == item_id)
            except:
                index = -1
            hit = int(index in range(0, topn))
            return hit, index

    def evaluate_model_for_user(self, model, person_id):
        #Getting the items in test set
        interacted_values_testset = interactions_test_indexed_df.loc[person_id]
        if type(interacted_values_testset['contentId']) == pd.Series:
            person_interacted_items_testset = set(interacted_values_testset['contentId'])
        else:
            person_interacted_items_testset = set([int(interacted_values_testset['contentId'])])  
        interacted_items_count_testset = len(person_interacted_items_testset) 

        #Getting a ranked recommendation list from a model for a given user
        person_recs_df = model.recommend_items(person_id, 
                                               items_to_ignore=get_items_interacted(person_id, 
                                                                                    interactions_train_indexed_df), 
                                               topn=10000000000)

        hits_at_5_count = 0
        hits_at_10_count = 0
        #For each item the user has interacted in test set
        for item_id in person_interacted_items_testset:
            #Getting a random sample (100) items the user has not interacted 
            #(to represent items that are assumed to be no relevant to the user)
            non_interacted_items_sample = self.get_not_interacted_items_sample(person_id, 
                                                                          sample_size=EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS, 
                                                                          seed=item_id%(2**32))

            #Combining the current interacted item with the 100 random items
            items_to_filter_recs = non_interacted_items_sample.union(set([item_id]))

            #Filtering only recommendations that are either the interacted item or from a random sample of 100 non-interacted items
            valid_recs_df = person_recs_df[person_recs_df['contentId'].isin(items_to_filter_recs)]                    
            valid_recs = valid_recs_df['contentId'].values
            #Verifying if the current interacted item is among the Top-N recommended items
            hit_at_5, index_at_5 = self._verify_hit_top_n(item_id, valid_recs, 5)
            hits_at_5_count += hit_at_5
            hit_at_10, index_at_10 = self._verify_hit_top_n(item_id, valid_recs, 10)
            hits_at_10_count += hit_at_10

        #Recall is the rate of the interacted items that are ranked among the Top-N recommended items, 
        #when mixed with a set of non-relevant items
        recall_at_5 = hits_at_5_count / float(interacted_items_count_testset)
        recall_at_10 = hits_at_10_count / float(interacted_items_count_testset)

        person_metrics = {'hits@5_count':hits_at_5_count, 
                          'hits@10_count':hits_at_10_count, 
                          'interacted_count': interacted_items_count_testset,
                          'recall@5': recall_at_5,
                          'recall@10': recall_at_10}
        return person_metrics

    def evaluate_model(self, model):
        #print('Running evaluation for users')
        people_metrics = []
        for idx, person_id in enumerate(list(interactions_test_indexed_df.index.unique().values)):
            #if idx % 100 == 0 and idx > 0:
            #    print('%d users processed' % idx)
            person_metrics = self.evaluate_model_for_user(model, person_id)  
            person_metrics['_person_id'] = person_id
            people_metrics.append(person_metrics)
        print('%d users processed' % idx)

        detailed_results_df = pd.DataFrame(people_metrics) \
                            .sort_values('interacted_count', ascending=False)
        
        global_recall_at_5 = detailed_results_df['hits@5_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        global_recall_at_10 = detailed_results_df['hits@10_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        
        global_metrics = {'modelName': model.get_model_name(),
                          'recall@5': global_recall_at_5,
                          'recall@10': global_recall_at_10}    
        return global_metrics, detailed_results_df
    
model_evaluator = ModelEvaluator()   